# Generate Segmentation Images based on NetSurfaceProblem

In [1]:
%matplotlib qt

import time
import copy
import numpy as np
from skimage.filters import gaussian

from tifffile import imread, imsave
import _pickle as pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from Netsurface3d import NetSurf3d
from data4d import Data4d
from stitch_surfaces import StitchSurfaces
from netsurface3d_radial import NetSurf3dRadial
from data4d_radial import Data4dRadial
from Visualization import Visualization
from CrossSection import CrossSection

# Load and Initialize

In [2]:
#filenames of images and of mask have to be in same order

#filenames = [ '/Users/wissmann/BobSeg/FlySegCropped.tif']
#filenames = [ '/Users/wissmann/BobSeg/Cell_croppedModel-0.0.1-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1)-1.tif']
#filenames = [ '/Users/wissmann/BobSeg/Vecad-flidsRed-48hpf.lif - fish1-48hpf-1.tif']
#filenames_mask = np.array([['/Users/wissmann/BobSeg/fish_mask.tif','/Users/wissmann/BobSeg/fish_mask_y_end.tif',None]])
filenames= ['/Users/wissmann/BobSeg/20180718_ecadgfp_0hAPF_disc05_cropped.tif']
filenames_mask = np.array([['/Users/wissmann/BobSeg/wing_mask_z.tif','/Users/wissmann/BobSeg/wing_mask_y.tif',None]])
#filenames = [ '/Users/wissmann/BobSeg/easy2surfaces.tif']
#filenames_mask = [ '/Users/wissmann/BobSeg/easymask.tif']
#filenames = [ '/Users/wissmann/BobSeg/HyperStack.tif']
#filenames = [ '/Users/wissmann/BobSeg/HyperStack3.tif']
#filenames = [ '/Users/wissmann/BobSeg/debugdx.tif']
#filenames = ['/Users/wissmann/BobSeg/nonquadratictest.tif']

In [3]:
#Methods : [' z  ,  y ,  x , rad']
chosen  =  [  1  ,  1 ,  0 ,  0  ]

#if you change chosen, Kernel restart is required

In [17]:
# Parameters for Netsurf with Base Graph
#             [ z, y, x]
K           = [50,700,30]                 
max_delta_k = [4 , 3, 2]
dx          = [20,20, 4]
dy          = [20, 4,10]
surfaces    = [ 1, 2, 2]
min_dist    = [ 6, 20, 2]                 #In terms of K
max_dist    = [50,100,50]                 #In terms of K
plot_base_graph =False
#create column from.. to..
c_from      = [ 0, 320, 0] 
c_to        = [ 0, 1180, 0]          #TODO (atm always -top- border of image)

In [18]:
# Parameters for Radial Netsurf
K_Rad            = 40
max_delta_k_rad  = 3
max_rs           = (800,740,220)                                   #In px (x,y,z)
min_rs           = (1,1,1)                                         #In px (x,y,z)
centers          = [(500,500,100),(200,200,100)]                   #In px (x,y,z)

# Segmentation

In [ ]:
data={}
Bg={}
Ra={}

#Base graph
for i in range(3):
    if chosen[i]==1:
        data[i]=Data4d(filenames, axis=i, filenames_mask=filenames_mask[:,i], pixelsize=(1.,1.,1.), silent=False, plane=None )
        data[i].set_seg_params(K[i],max_delta_k[i],dx[i],dy[i],surfaces[i],min_dist[i],max_dist[i])
        Bg[i] = data[i].init_object("Bg_%s"%i)
        data[i].add_object_at( Bg[i], c_from[i], c_to[i], frame=0, frame_to=0, segment_it=True, plot_base_graph=plot_base_graph )
    else: data[i]=None
        
#Radial
if chosen[3]==1:
    data[3]=Data4dRadial( filenames, pixelsize=(1.,1.,1.), silent=False )
    data[3].set_seg_params(K_Rad,max_delta_k_rad)
    for center in range(len(centers)):
        Ra[center]=data[3].init_object("Ra_%s"%center)
        data[3].add_object_at(Ra[center], min_rs, max_rs, frame=0, seed=centers[center], frame_to=0, segment_it=True )
else: data[3]=None
    
data[5]=Data4d(filenames, axis=0, filenames_mask=filenames_mask[:,0], pixelsize=(1.,1.,1.), silent=True, plane=None )
images=data[5].images

Dimensions (z,y,x) of frame 0 :  (151, 1437, 1479)
Searching for surface(s) along z direction
Dimensions (y,x) of mask 0 :  (1437, 1479)
Added appearance for "Bg_0" in frame 0
Number of columns: 3115
Number of Surfaces: 1
Done with surface 0
Elapsed time is 8.899250984191895 seconds.
      Optimum energy:  5025846.0
      Nodes in/out:  83622 72128
Dimensions (z,y,x) of frame 0 :  (151, 1437, 1479)
Searching for surface(s) along y direction
Changing y and z axis
New dimensions (z,y,x) of frame 0 : (1437, 151, 1479)
Dimensions (y,x) of mask 0 :  (151, 1479)
Added appearance for "Bg_1" in frame 0
Number of columns: 1717
Number of Surfaces: 2
Done with surface 0
Elapsed time is 63.42220091819763 seconds.


# Visualization
    Colour code for both 3d visualization and 2d cross-section plots: 
        Base graph (x,y,z): (green,blue,gold)
        Radial            : red

In [ ]:
vis = Visualization(data, images)

### 3D Visualization    
     
    show_single: shows all meshes from all chosen netsurf possibilities
    show_union: shows one union mesh of the single meshes in new window 
          (only makes sense in case of one main mesh and several small for with masks specified areas) TODO
    export: exports meshes as .obj files e.g. to be opened in blender
    stitch: in case of base graph with more than 1 surface, stitches the surfaces together to simulate volume
        setting stitch to True applies for all base graph netsurfs

In [ ]:
vis.show_frame(f=0, stackUnits=[1.,1.,1.], show_single=True, show_union=False, export=False, stitch=False)

### 2D Cross-Sections
    plane_normal: so far only axis-parallel vectors possible

In [15]:
from_slice = 520
to_slice   = 520
levels  = np.arange(from_slice, to_slice+1, step=1)

In [16]:
vis.show_sections(f=0,plane_orig=[0,0,0], plane_normal=[1,0,0], num_slices=levels, show_image=True) #in [x,y,z]

(1, 2, 3115, 3)
(1, 2, 1717, 3)
